# Part 1: Modeling

In [11]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from time import time

In [37]:
claim_df = pd.read_csv(r'C:\Users\Pat\Documents\Work\Projects\Trupanion Assignment\claimdata.csv', encoding = 'ISO-8859-1')
pet_df = pd.read_csv(r'C:\Users\Pat\Documents\Work\Projects\Trupanion Assignment\petdata.csv', encoding = 'ISO-8859-1')

pet_df

,PetId,EnrollDate,CancelDate,Species,Breed,AgeAtEnroll
0,0,2015-08-11,2018-07-13,Dog,Mixed Breed,5 years old
1,1,2014-02-15,2016-01-25,Cat,Mixed Breed,3 years old
2,2,2014-12-20,NaN,Dog,French Bulldog,8 weeks to 12 months old
3,3,2015-01-09,2015-01-14,Dog,Mixed Breed,1 year old
4,4,2014-07-20,NaN,Cat,Mixed Breed,8 weeks to 12 months old
...,...,...,...,...,...,...
9995,9995,2019-02-07,NaN,Cat,Mixed Breed,6 years old
9996,9996,2016-12-25,2018-12-31,Cat,Mixed Breed,8 weeks to 12 months old
9997,9997,2015-01-14,2015-03-11,Dog,Mixed Breed,8 weeks to 12 months old
9998,9998,2015-11-15,NaN,Cat,Mixed Breed,8 weeks to 12 months old


## Exploratory Data Analysis

**Here I'm mapping the AgeAtEnroll field to be a numerical value, since I feel like it's more of an Ordinal variable.**

In [ ]:
equiv1 = {
'0-7 weeks old':'0'
,'1 year old':'1'
,'10 years old':'10'
,'11 years old':'11'
,'12 years old':'12'
,'13 years old':'13'
,'2 years old':'2'
,'3 years old':'3'
,'4 years old':'4'
,'5 years old':'5'
,'6 years old':'6'
,'7 years old':'7'
,'8 weeks to 12 months old':'0.153846153846154'
,'8 years old':'8'
,'9 years old':'9'
}

pet_df['AgeAtEnroll'] = pet_df['AgeAtEnroll'].map(equiv1)
pet_df['AgeAtEnroll'] = pet_df['AgeAtEnroll'].astype(float)

**Now I'm creating a field for the duration of the policy, either as of the cancellation date or as of June 30th 2019.**

In [38]:
pet_df['CancelDate'] = pet_df.CancelDate.replace(np.NaN, '2019-06-30')
pet_df['CancelDate'] =  pd.to_datetime(pet_df['CancelDate'])
pet_df['EnrollDate'] =  pd.to_datetime(pet_df['EnrollDate'])
pet_df['Policy_Days'] = pet_df['CancelDate'] - pet_df['EnrollDate']
pet_df['Policy_Days'] = pet_df['Policy_Days'].dt.days.astype(int)
pet_df['Policy_Days'] = pet_df.Policy_Days.replace(0, 1)


pet_df

,PetId,EnrollDate,CancelDate,Species,Breed,AgeAtEnroll,Policy_Days
0,0,2015-08-11,2018-07-13,Dog,Mixed Breed,5.000000,1067
1,1,2014-02-15,2016-01-25,Cat,Mixed Breed,3.000000,709
2,2,2014-12-20,2019-06-30,Dog,French Bulldog,0.153846,1653
3,3,2015-01-09,2015-01-14,Dog,Mixed Breed,1.000000,5
4,4,2014-07-20,2019-06-30,Cat,Mixed Breed,0.153846,1806
...,...,...,...,...,...,...,...
9995,9995,2019-02-07,2019-06-30,Cat,Mixed Breed,6.000000,143
9996,9996,2016-12-25,2018-12-31,Cat,Mixed Breed,0.153846,736
9997,9997,2015-01-14,2015-03-11,Dog,Mixed Breed,0.153846,56
9998,9998,2015-11-15,2019-06-30,Cat,Mixed Breed,0.153846,1323


**Aggregate the claim data to the policy-level**

In [39]:
claim_df2 = claim_df.groupby('PetId')['ClaimAmount'].sum().reset_index()

In [40]:
dummy_column_names = list(pd.get_dummies(pet_df['Species'], prefix = 'Species', drop_first=False).columns)
dummy_column_names = dummy_column_names + list(pd.get_dummies(pet_df['Breed'], prefix = 'Breed', drop_first=True).columns)

pet_df= pd.get_dummies(pet_df, prefix = ['Species', 'Breed'])

# print(dummy_column_names)
pet_df
# pet_df[['Species', 'Breed']]

,PetId,EnrollDate,CancelDate,AgeAtEnroll,Policy_Days,Species_Cat,Species_Dog,Breed_Chihuahua,Breed_French Bulldog,Breed_Golden Retriever,Breed_Great Dane,Breed_Mixed Breed,Breed_Ragdoll
0,0,2015-08-11,2018-07-13,5.000000,1067,0,1,0,0,0,0,1,0
1,1,2014-02-15,2016-01-25,3.000000,709,1,0,0,0,0,0,1,0
2,2,2014-12-20,2019-06-30,0.153846,1653,0,1,0,1,0,0,0,0
3,3,2015-01-09,2015-01-14,1.000000,5,0,1,0,0,0,0,1,0
4,4,2014-07-20,2019-06-30,0.153846,1806,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,2019-02-07,2019-06-30,6.000000,143,1,0,0,0,0,0,1,0
9996,9996,2016-12-25,2018-12-31,0.153846,736,1,0,0,0,0,0,1,0
9997,9997,2015-01-14,2015-03-11,0.153846,56,0,1,0,0,0,0,1,0
9998,9998,2015-11-15,2019-06-30,0.153846,1323,1,0,0,0,0,0,1,0


**Now I'll make a field for the Reported per Month (on average) for each policy. This will be my target variabe.**

In [41]:
pet_df2 = pd.merge(pet_df, claim_df2, on='PetId', how = 'left')
pet_df2['ClaimAmount'] = pet_df2.ClaimAmount.replace(np.NaN, 0)
pet_df2['Reported per Day'] = pet_df2['ClaimAmount'] / pet_df2['Policy_Days']
pet_df2['Reported per Month'] = pet_df2['Reported per Day'] * 31

pet_df2

,PetId,EnrollDate,CancelDate,AgeAtEnroll,Policy_Days,Species_Cat,Species_Dog,Breed_Chihuahua,Breed_French Bulldog,Breed_Golden Retriever,Breed_Great Dane,Breed_Mixed Breed,Breed_Ragdoll,ClaimAmount,Reported per Day,Reported per Month
0,0,2015-08-11,2018-07-13,5.000000,1067,0,1,0,0,0,0,1,0,0.00,0.000000,0.000000
1,1,2014-02-15,2016-01-25,3.000000,709,1,0,0,0,0,0,1,0,0.00,0.000000,0.000000
2,2,2014-12-20,2019-06-30,0.153846,1653,0,1,0,1,0,0,0,0,2643.44,1.599177,49.574495
3,3,2015-01-09,2015-01-14,1.000000,5,0,1,0,0,0,0,1,0,0.00,0.000000,0.000000
4,4,2014-07-20,2019-06-30,0.153846,1806,1,0,0,0,0,0,1,0,1356.79,0.751268,23.289308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,2019-02-07,2019-06-30,6.000000,143,1,0,0,0,0,0,1,0,148.71,1.039930,32.237832
9996,9996,2016-12-25,2018-12-31,0.153846,736,1,0,0,0,0,0,1,0,903.92,1.228152,38.072717
9997,9997,2015-01-14,2015-03-11,0.153846,56,0,1,0,0,0,0,1,0,0.00,0.000000,0.000000
9998,9998,2015-11-15,2019-06-30,0.153846,1323,1,0,0,0,0,0,1,0,7657.43,5.787929,179.425797


In [42]:
print(dummy_column_names)

['Species_Cat', 'Species_Dog', 'Breed_French Bulldog', 'Breed_Golden Retriever', 'Breed_Great Dane', 'Breed_Mixed Breed', 'Breed_Ragdoll']


In [43]:
pet_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 16 columns):
PetId                     10000 non-null int64
EnrollDate                10000 non-null datetime64[ns]
CancelDate                10000 non-null datetime64[ns]
AgeAtEnroll               10000 non-null float64
Policy_Days               10000 non-null int32
Species_Cat               10000 non-null uint8
Species_Dog               10000 non-null uint8
Breed_Chihuahua           10000 non-null uint8
Breed_French Bulldog      10000 non-null uint8
Breed_Golden Retriever    10000 non-null uint8
Breed_Great Dane          10000 non-null uint8
Breed_Mixed Breed         10000 non-null uint8
Breed_Ragdoll             10000 non-null uint8
ClaimAmount               10000 non-null float64
Reported per Day          10000 non-null float64
Reported per Month        10000 non-null float64
dtypes: datetime64[ns](2), float64(4), int32(1), int64(1), uint8(8)
memory usage: 742.2 KB


A note on reported per month, I'm not taking into account any potential seasonality characteristics pet claims might have. I don't knwo if claims in July are different than claims in December, but here I'm just assuming they're the same.

## Creating the Random Forest Regressor

In [44]:
X = pet_df2[['AgeAtEnroll'] + dummy_column_names]
Y = pet_df2['Reported per Month']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

In [45]:
rfr = ensemble.RandomForestRegressor(n_estimators = 500, random_state = 465)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 4],
              "min_samples_split": [2, 3, 10]}


# # # run grid search
grid_search = GridSearchCV(rfc, param_grid=param_grid, cv=5, iid=False)
start = time()
grid_search.fit(X_train, y_train)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

y_pred = grid_search.predict(X_test)

rfr.score(X_test,y_test)

GridSearchCV took 131.26 seconds for 18 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.009 (std: 0.007)
Parameters: {'max_depth': 3, 'max_features': 4, 'min_samples_split': 3}

Model with rank: 2
Mean validation score: 0.009 (std: 0.007)
Parameters: {'max_depth': 3, 'max_features': 4, 'min_samples_split': 2}

Model with rank: 3
Mean validation score: 0.009 (std: 0.007)
Parameters: {'max_depth': 3, 'max_features': 3, 'min_samples_split': 3}



0.01734457582165405

In [51]:
y_pred2 = grid_search.predict(X)
pet_df2['RF July Predicted Claims'] = y_preds2

pet_df2[['PetId', 'Reported per Month', 'RF July Predicted Claims']]

,PetId,Reported per Month,RF July Predicted Claims
0,0,0.000000,62.939012
1,1,0.000000,34.014917
2,2,49.574495,78.410265
3,3,0.000000,45.552756
4,4,23.289308,21.643927
...,...,...,...
9995,9995,32.237832,47.054609
9996,9996,38.072717,21.643927
9997,9997,0.000000,41.874894
9998,9998,179.425797,21.643927


The model's very low R Squared is kind of concerning, but I played around with the parameters and tried a different kind of model with no luck. I personally have never made use of Machine Learning in an Insurance environment (we tried once at my place of employment but it didn't end up going anywhere). I'm not sure if that's the nature of the industry, this particular example, or if I just can't seem to make a useful model here. That being said, looking at the Reported per Month vs. the predicted values it doesn't seem to be incredible off the mark. So maybe the model isn't entirely useless. If I were to go back and try again, I might train the model at the Policy-Month Level instead of the less granular Policy Level. In addition, it might behoove me to cap any potentially extreme months for a policy if I deem it not indicative of future losses.

# Part 2: Short Answer

Using a proper claims management system is critical. Do your rewsearch, think about what systems you may have used in the past, talk with the claims folk to get their opinion, talk with IT to make sure you're compliant with their rules and guidelines, and consult with the proper figures on what system works best to fit your goals. My company uses ClaimCenter and I've heard no real compaints about it.

You also need to make sure you're capturing the necessary information. You'd want the obvious fields, such as paid, case reserves, ALAE, ULAE etc. You also need to be clear on what additional fields you'd need. For example, the location of the claim itself (you probably want that for policy, like the state of residence for the pet), a description of the claim (do you want a discrete list of options or a free form text field?), and either date of loss, reported date, or both (depending on the policy type).

Lastly you'll need to consult with your Data Engineers to make sure they won't have any issues getting the data from the claims syystem into their databases and to make sure they have everything they need on their end.